In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ConversationBufferWindowMemory
import re

def get_user_info():
    return {
        email:"test@test.com"
    }

# 이메일에 해당하는 이전 세션 정보 가져오기
# get_user_session_history


# 만약 세션 정보가 없다면 신규로 id를 발급 대기
# if len(session_info) == 0: generate_session_id()

# 세션 정보가 있다해도 새로운 채팅으로 랜딩되는 것이기 때문에
# 사용자의 새로운 대화가 시작되면 신규로 session_id 발급

# sesstion id 그리고 이전 대화 내용 저장 필요 -> 데이터베이스 저장

# 현재는 store로 대체
store = {}

# lens는 대화를 여러개 두지 않는 상황
# login 정보 확인 -> 이전 세션 ID 조회 -> 대화 내역 출력

# 세션 히스토리를 가져오기
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]

    # session_id로 조회가 된다면 직전 3개의 message만 가져옴
    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id],
        k=3,
        return_messages=True,
    )
    assert len(memory.memory_variables) == 1
    key = memory.memory_variables[0]
    messages = memory.load_memory_variables({})[key]
    store[session_id] = InMemoryChatMessageHistory(messages=messages)
    return store[session_id]

# 기존 데이터베이스 정보 및 db 가져오기
def callDatabase():
    db = SQLDatabase.from_uri("sqlite:///./sql_app.db")
    db_info = db.get_table_info()
    db_table_name = db.get_usable_table_names()
    return db, db_info, db_table_name

def get_table_info():
    db = SQLDatabase.from_uri("sqlite:///./sql_app.db")
    table_info = db.get_table_info()
    return table_info 

def makeTemplate():
    template = """
    당신은 매우 유능한 데이터분석가입니다.
    
    먼저 다음의 지시사항을 반드시 준수하세요.
    --- START OF CONDITIONS ---
    - 작업 순서를 반드시 지킬 것.
    - 질문의 의도를 정확히 반영할 것.
    --- END OF CONDITIONS ---
    
    그런 다음 데이터베이스 정보를 확인하세요.
    --- START OF DATABASE INFO ---
    {db_info}
    --- END OF DATABASE INFO ---
    
    이제 사용자의 질문을 확인하고, 순서에 따라 작업을 진행해주세요.
    --- START OF QUESTION ---
    {question}
    --- END OF QUESTION ---
    
    --- START OF TASK ---
    빈 배열이 주어집니다. 이 배열에는 다음의 내용을 순서대로 담아주세요.
    1. 질문이 데이터베이스를 활용해 쿼리를 생성할 수 있다면 쿼리 생성을 위한 전략과 실행시간, 접근방법을 문자열로 배열에 추가 하고 그렇지 않다면 자연스러운 응답을 문자열로 배열에 추가하세요.
    2. 질문이 데이터베이스를 활용해 쿼리를 생성할 수 있다면 생성된 쿼리를 문자열로 배열에 추가하고 그렇지 않으면 빈 문자열을 담아주세요.
    --- END OF TASK ---
    
    작업을 완료하면 배열을 반환해주세요.
    배열외의 내용은 모두 무시됩니다.
    
    백틱(`)을 사용한 코드 블록은 작성하지 마세요.
    오직 배열만 반환해주세요.
    """
    prompt = PromptTemplate.from_template(template)
    return prompt


def callLLM(prompt):
    settings = Settings()

    # API_KEY = settings.API_KEY
    # API_BASE = settings.API_BASE

    API_KEY = '152d00e9-bfc6-4097-9f75-91c91d42898a'
    API_BASE = 'http://43.202.9.204:8080/aihub/v2/sandbox'
    
    llm = ChatOpenAI(
        model="gpt-4o",
        openai_api_key=API_KEY,
        openai_api_base=API_BASE,
    )

    chain = RunnableWithMessageHistory(llm, get_session_history)
    response  = chain.invoke(
        "Hi I'm Bob.",
        config={"configurable": {"session_id": "1"}},
    )  # session_id determines thread

    return response


def findSQL(answer):
    pattern = r"sql\n(.*?);"
    match = re.search(pattern, answer, re.DOTALL)

    if match:
        sql_query = match.group(1)
        return sql_query
    else:
        return -1